https://www.microsoft.com/en-us/research/project/project-malmo/

http://gym.openai.com/

In [1]:
#pip install gym

In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import random
import astar as a
import maze as Maze
import agent as Agent

In [3]:
def run_maze(maze, robot, games, max_steps, starting_grid):
    moveHistory = []      # keep track of the number of moves.
    for i in range(games): # number of games
        if i % 1000 == 0:
            print(i)      # just to know is running.
        while not maze.isGameOver():     # while the game is not over
            state, _ = maze.getStateAndReward()   # gets state and reward
            action = robot.chooseAction(state, maze.allowedStates[state]) # update the actions
            maze.updateMaze(action)
            state, reward = maze.getStateAndReward()
            robot.updateStateHistory(state, reward)
            if maze.steps > max_steps:                  # cut prematurely to avoid infinite looping
                x, y = maze.get_matrix().shape
                maze.robotPosition = (x - 1, y - 1)
        robot.learn()                              # learns
        moveHistory.append(maze.steps)             # keep tracking of steps
        maze = Maze.Maze(starting_grid)               # resets the maze but not the agent.
    return moveHistory

# Generate a valid maze

In [4]:
## generate matrix
def gen_matrix(size, wall_ratio):
    size = size
    total = size[0] * size[0]
    wall_nodes = int(wall_ratio * total) # 1s
    path_nodes = total - wall_nodes # 0s

    my_list = [0]*path_nodes + [1]*wall_nodes
    random.shuffle(my_list)

    temp = 0
    matrix = np.zeros(size)
    start = 0
    end = 5
    my_list = np.array(my_list)
    matrix = my_list.reshape((size))
    # Force a start and endnode
    matrix[0,0] = 0
    matrix[size[0]-1, size[0]-1] = 0
    
    visited = np.zeros(size, dtype=int)
    return matrix, visited

In [5]:
def gen_valid_matrix(size, wall_ratio):
    matrix = []
    res = None
    while(res==None):
        matrix, visit =  gen_matrix(size, wall_ratio)
        walls = [tuple(coord) for coord in np.argwhere(matrix == 1)]
        star = a.AStar()
        star.init_grid(size[0], 
                       size[1], 
                       walls, 
                       (0,0), 
                       (size[0]-1, size[0]-1)
                      )
        res = star.solve()
    matrix[0,0] = 2
    return matrix

In [6]:
input_maze = gen_valid_matrix((10,10), 0.5)

In [7]:
input_maze

array([[2, 1, 1, 0, 1, 1, 1, 0, 0, 1],
       [0, 0, 1, 1, 1, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 1, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 1, 1, 0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0]])

In [8]:
maze = Maze.Maze(input_maze)
for y,row in enumerate(maze.get_matrix()):
    print(row)

[2 1 1 0 1 1 1 0 0 1]
[0 0 1 1 1 1 0 1 0 1]
[0 0 0 1 1 1 0 0 0 0]
[0 0 1 1 0 1 0 1 1 0]
[0 1 1 1 1 1 1 1 1 1]
[0 0 0 0 1 0 0 0 1 1]
[1 0 1 0 0 0 0 0 1 0]
[1 0 1 1 0 0 1 0 1 0]
[0 0 1 1 0 1 1 1 1 0]
[0 1 1 1 0 0 0 0 0 0]


In [9]:
maze.constructAllowedStates()

In [ ]:
input_maze = gen_valid_matrix((10,10), 0.5)
maze = Maze.Maze(input_maze)
robot = Agent.Agent(maze, alpha=0.01, randomFactor=0.9)
history = run_maze(maze, robot, 5000, 5000, input_maze)
 
#maze = Maze.Maze(input_maze)
#robot = Agent.Agent(maze, alpha=0.1, randomFactor=0.25)
#history_2 = run_maze(maze, robot)

0
1000
2000
3000
4000


In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(211)
plt.semilogy(history, 'b--')
plt.legend(['alpha=0.1'])
